In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/infor-mail/MAIL.csv
/kaggle/input/mail-test/MAIL_test.csv


In [16]:
df = pd.read_csv('/kaggle/input/infor-mail/MAIL.csv',header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,0,-1.0,-1.00,-1.000000,-1.0,-1.00000,-1.000000,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-0.148221,-1.000000,-1.000000,-1.000000,-0.754045,-1.000000,-0.718446,-1.0
1,0,-1.0,-1.00,-1.000000,-1.0,-0.82691,-0.632182,-1.0,-1.0,-0.720307,...,-1.0,-1.0,-0.554151,-1.000000,-0.936759,-0.640316,-1.000000,-0.312254,-1.000000,-1.0
2,0,-1.0,-0.84,-0.820690,-1.0,-1.00000,-1.000000,-1.0,-1.0,-0.635001,...,-1.0,-1.0,-0.608332,-0.916667,-1.000000,-0.763021,-1.000000,-1.000000,-1.000000,-1.0
3,0,-1.0,-1.00,-0.886152,-1.0,-1.00000,-0.796825,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-0.387624,-1.000000,-0.895765,-0.703583,-1.000000,-0.811075,-0.716612,-1.0
4,0,-1.0,-1.00,-0.774452,-1.0,-1.00000,-0.798742,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-1.000000,-0.894040,-0.894040,-1.000000,-0.496689,-1.000000,-1.000000,-1.0


In [17]:
df.shape

(3587, 401)

In [18]:
df.values[df.isnull().sum(axis=1) >= 1]

array([], shape=(0, 401), dtype=float64)

In [19]:
pel_class = np.unique(df[0])
pel_class

array([0, 1])

In [20]:
pel_value = [sum(df[0]==pel_class[0]),
             sum(df[0]==pel_class[1])]
pel_value

[1204, 2383]

In [21]:
y = df[0].values
X = df.drop([0],axis = 1).values

In [22]:
X = np.array(X)
y = np.array(y)

In [23]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X, y = sm.fit_resample(X, y)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [26]:
X_train_norm.shape

(3812, 400)

In [27]:
from sklearn.decomposition import PCA
fr = PCA(n_components=400)
fr.fit(X_train_norm)
X_train_norm = fr.transform(X_train_norm)
X_test_norm = fr.transform(X_test_norm)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators' : [10,50,100],
    'min_samples_leaf' : [0.25,0.5,1],
    'max_features': ['auto', 'sqrt'],
    'criterion' : ['gini','entropy']
}
clf = GridSearchCV(RandomForestClassifier(random_state=0),params, cv = 5)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 100}
10CV accuracy : 82.60743783173892


In [29]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

Test accuracy : 80.81761006289308


In [30]:
df_test = pd.read_csv('/kaggle/input/infor-mail/MAIL.csv',header = None)
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,0,-1.0,-1.00,-1.000000,-1.0,-1.00000,-1.000000,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-0.148221,-1.000000,-1.000000,-1.000000,-0.754045,-1.000000,-0.718446,-1.0
1,0,-1.0,-1.00,-1.000000,-1.0,-0.82691,-0.632182,-1.0,-1.0,-0.720307,...,-1.0,-1.0,-0.554151,-1.000000,-0.936759,-0.640316,-1.000000,-0.312254,-1.000000,-1.0
2,0,-1.0,-0.84,-0.820690,-1.0,-1.00000,-1.000000,-1.0,-1.0,-0.635001,...,-1.0,-1.0,-0.608332,-0.916667,-1.000000,-0.763021,-1.000000,-1.000000,-1.000000,-1.0
3,0,-1.0,-1.00,-0.886152,-1.0,-1.00000,-0.796825,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-0.387624,-1.000000,-0.895765,-0.703583,-1.000000,-0.811075,-0.716612,-1.0
4,0,-1.0,-1.00,-0.774452,-1.0,-1.00000,-0.798742,-1.0,-1.0,-1.000000,...,-1.0,-1.0,-1.000000,-0.894040,-0.894040,-1.000000,-0.496689,-1.000000,-1.000000,-1.0


In [31]:
yt = df_test[0].values
Xt = df_test.drop([0],axis = 1).values

In [33]:
X_test = scaler.transform(Xt)
X_test_n = fr.transform(X_test)

In [34]:
y_predict = clf.predict(X_test_n)
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))

Test accuracy : 95.28854195706718
